In [5]:
import pandas as pd
import numpy as np

caminho = r'C:\Pedro\Python\Dissertação\Dados\Opções PETR4 - CALLs e PUTs - lista, pesquisa e cotações.xlsx'

df = pd.read_excel(caminho, header=1)

df


,Ticker,Vencimento,Dias úteis,Tipo,F.M.,Mod.,Strike,A/I/OTM,Dist. (%) do Strike,Último,...,Gamma,Theta ($),Theta (%),Vega,IQ,Coberto,Travado,Descob.,Tit.,Lanç.
0,PETRI220,19/09/2025,16,CALL,NaN,E,2147,ITM,-2997,868.0,...,0.0,-127.0,-15.0,0.0,57.0,1,1.1,12.2,4.0,7.0
1,PETRU220,19/09/2025,16,PUT,NaN,E,2147,OTM,-2997,1.0,...,0.0,0.0,0.0,0.0,173.0,0,259.4,1.132.600,19.0,11.0
2,PETRJ220,17/10/2025,36,CALL,NaN,A,2147,ITM,-2997,943.0,...,0.0,-125.0,-13.0,2.0,100.0,2.4,1.2,166.3,3.0,3.0
3,PETRV220,17/10/2025,36,PUT,NaN,E,2147,OTM,-2997,1.0,...,47.0,-14.0,-1397.0,2728.0,300.0,0,527.8,0,9.0,3.0
4,PETRK220,21/11/2025,60,CALL,NaN,E,2147,ITM,-2997,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1209,PETRV421,17/10/2025,36,PUT,NaN,E,3997,ITM,"+30,37",317.0,...,619.0,42.0,13.0,112800.0,100.0,0,0,2,1.0,1.0
1210,PETRK23,21/11/2025,60,CALL,NaN,E,3997,OTM,"+30,37",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1211,PETRW23,21/11/2025,60,PUT,NaN,E,3997,ITM,"+30,37",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1212,PETRA421,16/01/2026,96,CALL,NaN,A,3997,OTM,"+30,37",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Renomeando as colunas

In [6]:
df.columns

Index(['Ticker', 'Vencimento', 'Dias úteis', 'Tipo', 'F.M. ', 'Mod.', 'Strike',
       'A/I/OTM', 'Dist. (%) do Strike', 'Último', 'Var. (%)', 'Data/Hora',
       'Núm. de Neg.', 'Vol. Financeiro', 'Vol. Imp.', 'Delta', 'Gamma',
       'Theta ($)', 'Theta (%)', 'Vega', 'IQ', 'Coberto', 'Travado', 'Descob.',
       'Tit.', 'Lanç.'],
      dtype='object')

In [7]:
xa0_col = []

for col in df.columns:
    if '\xa0' in col:
        xa0_col.append(col)

xa0_col

['F.M.\xa0', 'Var.\xa0(%)']

In [8]:
nomes = {
    'Strike\xa0\xa0\xa0\xa0': 'K',
    'Dias úteis': 'tau (dias)',
    'Último': 'V_mkt',
    'Vol. Impl. (%)': 'vol. imp.',
    'Tipo': 'tipo',
    'Núm. de Neg.': 'trades'
}

colunas = list(nomes.values())
df = df.rename(columns=nomes)

df = df[colunas]
df['K'] = df['K'] / 100
df['vol. imp.'] = df['vol. imp.'] / 1000
df['trades'] = pd.to_numeric(df['trades'], errors='coerce')
calib_data = df.dropna(subset=colunas)

calib_data

KeyError: "['K', 'vol. imp.'] not in index"

In [ ]:
min_trades = 10

calib_data = calib_data[calib_data['trades'] >= min_trades]

calib_data

,K,tau (dias),V_mkt,vol. imp.,tipo,trades
3,21.47,36,1.0,0.424,PUT,100.0
9,21.72,16,1.0,0.560,PUT,20.0
11,21.72,36,2.0,0.412,PUT,56.0
19,21.97,36,2.0,0.414,PUT,11.0
95,23.72,16,3.0,0.511,PUT,23.0
...,...,...,...,...,...,...
1087,37.72,36,641.0,0.302,PUT,11.0
1107,37.97,36,664.0,0.300,PUT,14.0
1179,39.40,6,729.0,0.637,PUT,18.0
1197,39.72,60,766.0,0.295,PUT,16.0


In [ ]:
tau_min = 21
tau_max = 504

calib_data = calib_data[ (calib_data['tau (dias)'] >= tau_min) & (calib_data['tau (dias)'] <= tau_max)]

calib_data

,K,tau (dias),V_mkt,vol. imp.,tipo,trades
3,21.47,36,1.0,0.424,PUT,100.0
11,21.72,36,2.0,0.412,PUT,56.0
19,21.97,36,2.0,0.414,PUT,11.0
176,24.97,60,626.0,0.254,CALL,10.0
182,25.22,36,574.0,0.222,CALL,12.0
...,...,...,...,...,...,...
1078,37.47,36,1.0,0.215,CALL,56.0
1087,37.72,36,641.0,0.302,PUT,11.0
1107,37.97,36,664.0,0.300,PUT,14.0
1197,39.72,60,766.0,0.295,PUT,16.0


In [ ]:
min_price = 0.01

calib_data = calib_data[calib_data['V_mkt'] > min_price]

calib_data

,K,tau (dias),V_mkt,vol. imp.,tipo,trades
3,21.47,36,1.0,0.424,PUT,100.0
11,21.72,36,2.0,0.412,PUT,56.0
19,21.97,36,2.0,0.414,PUT,11.0
176,24.97,60,626.0,0.254,CALL,10.0
182,25.22,36,574.0,0.222,CALL,12.0
...,...,...,...,...,...,...
1078,37.47,36,1.0,0.215,CALL,56.0
1087,37.72,36,641.0,0.302,PUT,11.0
1107,37.97,36,664.0,0.300,PUT,14.0
1197,39.72,60,766.0,0.295,PUT,16.0


In [ ]:
min_mon = 0.75
max_mon = 1.25
spot = 30.91

calib_data['moneyness'] = spot / calib_data['K']

call_cond =(
    (calib_data['tipo'] == 'CALL') &
    (calib_data['moneyness'] >= min_mon) &
    (calib_data['moneyness'] <= 1)
)

put_cond = (
    (calib_data['tipo'] == 'PUT') &
    (calib_data['moneyness'] >= 1) &
    (calib_data['moneyness'] <= max_mon)
)

calib_data = calib_data[call_cond | put_cond].reset_index(drop=True)

In [ ]:
calib_data['tau (anos)'] = calib_data['tau (dias)'] / 252

# Paridade Put-Call: C - P = S - Ke^{-rtau(anos)}
r = 0.15

calib_data['equiv_price'] =np.where(
    calib_data['tipo'] == 'CALL',                # Condição: é uma CALL?
    calib_data['V_mkt'],                         # Se sim: usa o V_mkt original
    calib_data['V_mkt'] + spot - calib_data['K'] * np.exp(-r * calib_data['tau (anos)']) # Se não (é PUT): calcula C = P + S - K*exp(-r*T)
)

calib_data

,K,tau (dias),V_mkt,vol. imp.,tipo,trades,moneyness,tau (anos),equiv_price
0,27.22,36,15.0,0.291,PUT,15.0,1.135562,0.142857,19.267081
1,27.72,36,18.0,0.281,PUT,18.0,1.115079,0.142857,21.777681
2,27.97,36,21.0,0.276,PUT,29.0,1.105113,0.142857,24.532981
3,28.22,36,27.0,0.269,PUT,36.0,1.095322,0.142857,30.288281
4,28.47,36,26.0,0.263,PUT,16.0,1.085704,0.142857,29.043581
5,28.72,36,30.0,0.259,PUT,35.0,1.076253,0.142857,32.798882
6,28.97,36,32.0,0.254,PUT,30.0,1.066966,0.142857,34.554182
7,29.22,36,37.0,0.252,PUT,25.0,1.057837,0.142857,39.309482
8,29.22,60,58.0,0.254,PUT,11.0,1.057837,0.238095,60.715156
9,29.25,80,67.0,0.257,PUT,11.0,1.056752,0.317460,70.020214


In [ ]:

ABEV3,"R$ 12,28",
BBAS3,"R$ 21,12",
BBDC4,"R$ 16,48 (aprox.)",
ITUB4,"R$ 38,40 (aprox.)",
PETR4,"R$ 30,91",
VALE3,"R$ 56,08",

In [10]:
import calibration_cleaning as cc

teste = cc.calibration_cleaning('VALE3', 56.08)

teste

,K,tau (dias),V_mkt,vol. imp.,tipo,trades,moneyness,tau (anos),equiv_price
0,46.82,36,9.0,0.291,PUT,14.0,1.197779,0.142857,19.252613
1,48.29,80,38.0,0.265,PUT,12.0,1.161317,0.317460,48.035632
2,49.07,36,15.0,0.263,PUT,12.0,1.142857,0.142857,23.050314
3,49.32,36,16.0,0.258,PUT,33.0,1.137064,0.142857,23.805614
4,50.32,36,21.0,0.248,PUT,18.0,1.114467,0.142857,27.826815
5,50.82,36,27.0,0.241,PUT,51.0,1.103503,0.142857,33.337415
6,51.32,36,31.0,0.238,PUT,22.0,1.092751,0.142857,36.848015
7,51.82,36,34.0,0.233,PUT,24.0,1.082208,0.142857,39.358616
8,52.32,36,41.0,0.230,PUT,99.0,1.071865,0.142857,45.869216
9,52.82,36,51.0,0.225,PUT,83.0,1.061719,0.142857,55.379816
